# Introduction to Modeling Libraries in Python

## Interfacing Between pandas as Model Core

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [.01, -.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.],
})
data.to_numpy()

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [3]:
data2 = pd.DataFrame(data.to_numpy(), columns=['one', 'two', 'three'])
data2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [4]:
data3 = data.copy()
data3['strings'] = list('abcde')

data3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [5]:
data3.to_numpy()

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [6]:
model_columns = ['x0', 'x1']
data.loc[:, model_columns].to_numpy()

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [7]:
data['category'] = pd.Categorical(list('abaab'), categories=['a', 'b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [8]:
dummies = pd.get_dummies(data.category, prefix='category')
data_with_dummies = data.drop('category', axis='columns').join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


## Creating Model Descriptions with Patsy

In [9]:
!pip install statsmodels
import statsmodels

In [10]:
data = pd.DataFrame({
    'x0': np.arange(5) + 1,
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]
})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [11]:
import patsy

y, X = patsy.dmatrices('y ~ x0 + x1', data)
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [12]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [13]:
np.asarray(y), np.asarray(X)

(array([[-1.5],
        [ 0. ],
        [ 3.6],
        [ 1.3],
        [-2. ]]),
 array([[ 1.  ,  1.  ,  0.01],
        [ 1.  ,  2.  , -0.01],
        [ 1.  ,  3.  ,  0.25],
        [ 1.  ,  4.  , -4.1 ],
        [ 1.  ,  5.  ,  0.  ]]))

In [14]:
coeficients, residual, _, _ = np.linalg.lstsq(X, y)

C:\Users\William Costa\AppData\Local\Temp\ipykernel_12236\2056467099.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coeficients, residual, _, _ = np.linalg.lstsq(X, y)


In [15]:
coeficients

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [16]:
coeficients = pd.Series(coeficients.squeeze(), index=X.design_info.column_names)
coeficients

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

### Data Transformations in Patsy Formulas

In [17]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1) + 1', data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [18]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [19]:
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [3.1, -0.5, 0, 2.3],
    'y': [1,2,3,4]
})

new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [20]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

### Categorical Data and Patsy

In [21]:
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})

y, X = patsy.dmatrices('v2 ~ key1', data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [22]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [23]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [24]:
data['key2'] = data['key2'].replace({0: 'zero', 1: 'one'})

In [25]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [26]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

## Introduction to Statsmodels

### Estimating Linear Models

In [27]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [28]:
rng = np.random.default_rng(seed=12345)


def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * rng.standard_normal(*size)


N = 100
X = np.c_[
    dnorm(0, 0.4, size=N),
    dnorm(0, 0.6, size=N),
    dnorm(0, 0.2, size=N)
]

eps = dnorm(0, 0.1, size=N)
beta = [.1, .3, .5]

y = np.dot(X, beta) + eps
X[:5], y[:5]

(array([[-0.90050602, -0.18942958, -1.0278702 ],
        [ 0.79925205, -1.54598388, -0.32739708],
        [-0.55065483, -0.12025429,  0.32935899],
        [-0.16391555,  0.82403985,  0.20827485],
        [-0.04765129, -0.21314698, -0.04824364]]),
 array([-0.59952668, -0.58845445,  0.18563386, -0.00747657, -0.01537445]))

In [29]:
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.        , -0.90050602, -0.18942958, -1.0278702 ],
       [ 1.        ,  0.79925205, -1.54598388, -0.32739708],
       [ 1.        , -0.55065483, -0.12025429,  0.32935899],
       [ 1.        , -0.16391555,  0.82403985,  0.20827485],
       [ 1.        , -0.04765129, -0.21314698, -0.04824364]])

In [30]:
model = sm.OLS(y, X)
results = model.fit()

In [31]:
results.params

array([0.06681503, 0.26803235, 0.45052319])

In [32]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.469
Model:                            OLS   Adj. R-squared (uncentered):              0.452
Method:                 Least Squares   F-statistic:                              28.51
Date:                Mon, 27 Mar 2023   Prob (F-statistic):                    2.66e-13
Time:                        11:27:41   Log-Likelihood:                         -25.611
No. Observations:                 100   AIC:                                      57.22
Df Residuals:                      97   BIC:                                      65.04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0668      0.054      1.243      0.217      -0.040       0.174
x2             0.2680      0.042      6.313      0.000       0.184       0.352
x3             0.4505      0.068      6.605      0.000       0.315       0.586
==============================================================================
Omnibus:                        0.435   Durbin-Watson:                   1.869
Prob(Omnibus):                  0.805   Jarque-Bera (JB):                0.301
Skew:                           0.134   Prob(JB):                        0.860
Kurtosis:                       2.995   Cond. No.                         1.64
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [33]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])
data['y'] = y
data[:5]

,col0,col1,col2,y
0,-0.900506,-0.189430,-1.027870,-0.599527
1,0.799252,-1.545984,-0.327397,-0.588454
2,-0.550655,-0.120254,0.329359,0.185634
3,-0.163916,0.824040,0.208275,-0.007477
4,-0.047651,-0.213147,-0.048244,-0.015374


In [34]:
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()
results.params, results.tvalues

(Intercept   -0.020799
 col0         0.065813
 col1         0.268970
 col2         0.449419
 dtype: float64,
 Intercept   -0.652501
 col0         1.219768
 col1         6.312369
 col2         6.567428
 dtype: float64)

In [35]:
results.predict(data[:5])

0   -0.592959
1   -0.531160
2    0.058636
3    0.283658
4   -0.102947
dtype: float64

### Estimating Time Series Processes

In [36]:
init_x = 4
values = [init_x, init_x]

N = 1000
b0 = 0.8
b1 = -0.4

noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

In [37]:
from statsmodels.tsa.ar_model import AutoReg
MAXLAGS = 5

model = AutoReg(values, MAXLAGS)
results = model.fit()
results.params

array([ 0.02346612,  0.8096828 , -0.42865278, -0.03336517,  0.04267874,
       -0.05671529])

## Introduction to scikit-learn

In [41]:
import seaborn as sns
train = pd.read_csv(r'C:\Users\William Costa\Documents\repositories\data_science_basic_knowledge\resources\train.csv')
test = pd.read_csv(r'C:\Users\William Costa\Documents\repositories\data_science_basic_knowledge\resources\test.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [42]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [43]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [44]:
impute_value = train['Age'].median()
train['Age'] = train['Age'].fillna(impute_value)
test['Age'] = test['Age'].fillna(impute_value)

In [45]:
train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

In [54]:
predictors = ['Pclass', 'IsFemale', 'Age']
x_train = train[predictors].to_numpy()
x_test = test[predictors].to_numpy()
y_train = train['Survived'].to_numpy()
x_train[:5], y_train[:5]

(array([[ 3.,  0., 22.],
        [ 1.,  1., 38.],
        [ 3.,  1., 26.],
        [ 1.,  1., 35.],
        [ 3.,  0., 35.]]),
 array([0, 1, 1, 1, 0], dtype=int64))

In [50]:
!pip install scikit-learn
from sklearn.linear_model import LogisticRegression

In [51]:
model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [52]:
y_predict = model.predict(x_test)
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0], dtype=int64)

In [55]:
# Get mean if you have y_true -> (y_true == y_predict).mean()

In [56]:
from sklearn.linear_model import LogisticRegressionCV

modelcv = LogisticRegressionCV(Cs=10)
modelcv.fit(x_train, y_train)

LogisticRegressionCV()

In [58]:
from sklearn.model_selection import cross_val_score

model = LogisticRegression(C=10)
scores = cross_val_score(model, x_train, y_train, cv=4)
scores

array([0.77578475, 0.79820628, 0.77578475, 0.78828829])